<a href="https://colab.research.google.com/github/zorawarMahal/Spotify-Music-Recommender/blob/main/SpotifyMusicRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import base64

# storing clientID and Client Secret Key
CLIENT_ID = '202524f1de4c4506a1595ac47c86a4ba'
CLIENT_SECRET = 'e0652abb8cfe4f4d8e1bd96cf2c81662'

#base64 encodes the clientId and secret key
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

#request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
  access_token = response.json()['access_token']
  print("Access token obtained successfully.")
else:
  print("Error obtaining access token")
  exit()

Access token obtained successfully.


In [4]:
pip install spotipy

In [11]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [12]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'

# Call the funtion to get the music data from the playlist and store it in a dataframe
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,APT.,"ROSÉ, Bruno Mars",APT.,2IYQwwgxgOIn7t3iF6ufFD,5vNRhkKd0yEAg8suGBpjeY,97,2024-10-18,169917,False,https://open.spotify.com/track/5vNRhkKd0yEAg8s...,...,0.783,0,-4.477,0,0.2600,0.02830,0.000000,0.355,0.939,149.027
1,The Emptiness Machine,Linkin Park,The Emptiness Machine,6W0Gabv5f3ugnckc6YgfJQ,2PnlsTsOTLE5jnBnNe2K0A,90,2024-09-05,190428,True,https://open.spotify.com/track/2PnlsTsOTLE5jnB...,...,0.872,7,-3.344,1,0.0336,0.01560,0.000000,0.121,0.806,184.115
2,Not Like Us,Kendrick Lamar,Not Like Us,5JjnoGJyOxfSZUZtk2rRwZ,6AI3ezQ4o3HUoP6Dhudph3,88,2024-05-04,274192,True,https://open.spotify.com/track/6AI3ezQ4o3HUoP6...,...,0.472,1,-7.001,1,0.0776,0.01070,0.000000,0.141,0.214,101.061
3,Rah Tah Tah,"Tyler, The Creator",CHROMAKOPIA,0U28P0QVB1QRxpqp5IHOlH,5RePVWy39tLpHH0WwXgBsK,87,2024-10-28,165413,True,https://open.spotify.com/track/5RePVWy39tLpHH0...,...,0.878,1,-4.099,1,0.3100,0.00289,0.000000,0.381,0.514,82.557
4,Big Dawgs,"Hanumankind, Kalmi",Big Dawgs,6Yw4204wbgmpsGTzjXBhYD,0OA00aPt3BV10qeMIs3meW,86,2024-07-09,190667,True,https://open.spotify.com/track/0OA00aPt3BV10qe...,...,0.745,4,-3.202,0,0.1610,0.02350,0.000000,0.363,0.262,180.098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,REACT,"Switch Disco, Ella Henderson, Robert Miles",REACT,3opvHAj80TIQMZAg9a2FNv,1UPHCP5YeVfele4DMbdGyi,70,2023-01-13,201146,False,https://open.spotify.com/track/1UPHCP5YeVfele4...,...,0.899,6,-3.189,0,0.0556,0.03290,0.000000,0.186,0.358,125.992
96,Astronaut In The Ocean,Masked Wolf,Astronaut In The Ocean,7vus4Q8r5DS2Dl1JClxEsA,3Ofmpyhv5UAQ70mENzB277,69,2021-01-06,132780,True,https://open.spotify.com/track/3Ofmpyhv5UAQ70m...,...,0.695,4,-6.865,0,0.0913,0.17500,0.000000,0.150,0.472,149.996
97,Another Love - Tiësto Remix,"Tom Odell, Tiësto",Another Love (Tiësto Remix),1QltVGlNGbK94CKgUsYZGa,4Et6tUEO7biKxZEfJXpNj1,69,2022-05-13,185366,False,https://open.spotify.com/track/4Et6tUEO7biKxZE...,...,0.764,4,-4.685,0,0.0737,0.08360,0.087100,0.132,0.473,123.046
98,Tarantella,"Gabry Ponte, KEL",Tarantella,0nxNoLQWFwsRqTfLh7R0sq,4pBuhoBxgwBpkkOhfT6p6N,69,2024-03-29,145554,False,https://open.spotify.com/track/4pBuhoBxgwBpkkO...,...,0.861,9,-5.509,1,0.0781,0.13900,0.000026,0.332,0.426,140.072


In [13]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [15]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [16]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [17]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [18]:
import pandas as pd

def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]
    )

    new_entry = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index=True)

    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations

In [19]:
input_song_name = "I'm Good (Blue)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'I'm Good (Blue)':
                     Track Name                                     Artists  \
1                       Disease                                   Lady Gaga   
2                       KEEP UP                                     Odetari   
3  It's Not Right But It's Okay                            Mr. Belt & Wezol   
4         Rainfall (Praise You)                                   Tom Santa   
0                         REACT  Switch Disco, Ella Henderson, Robert Miles   

                     Album Name Release Date  Popularity  
1                       Disease   2024-10-25        86.0  
2          KEEP UP // FROSTBITE   2024-07-17        85.0  
3  It's Not Right But It's Okay   2024-02-23        78.0  
4         Rainfall (Praise You)   2022-02-18        71.0  
0                         REACT   2023-01-13        70.0  
